# Segmenting and Clustering Neighbourhoods in Toronto

##### The project includes scraping the Wikipedia page for the postal codes of Canada and then process and clean the data for the clustering. The clustering is carried out by K Means and the clusters are plotted using the Folium Library. The Boroughs containing the name 'Toronto' in it are first plotted and then clustered and plotted again.



<h3>Installing and Importing the required Libraries</h3>

In [1]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Solving environment: done

# All requested packages already installed.

Folium installed
Lib

<h3>Scraping the Wikipedia page for the table of postal codes of Canada</h3>

##### BeautifulSoup Library of Python is used for web scraping of table from the Wikipedia. The title of the webpage is printed to check if the page has been scraped successfully or not. Then the table of postal codes of Canada is printed.

In [11]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

data_html = BeautifulSoup(source.content)

# read the data into a Pandas Dataframe
soup = BeautifulSoup(str(data_html))


##### Creation of Coloumn Names:

In [34]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
      # Create three columns named as "PostalCode","Borough" & "Neighborhood"
        cell['Postal Code'] = row.p.text[:3] # store only first three letter from the test of <p> tab.(Ex: M3A )
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighbourhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        # here we replace some symbols like "(" , ")" , "/" from the neighborhood name(Ex: (Parkview Hill / Woodbine Gardens))
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
# compress some big borough name by smaller one
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


<h3>Data preprocessing and cleaning</h3>

In [35]:
# Dropping the rows where Borough is 'Not assigned'
df1 = df[df.Borough != 'Not assigned']

# Combining the neighbourhoods with same Postalcode
df2 = df1.groupby(['Borough','Postal Code'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)

# Replacing the name of the neighbourhoods which are 'Not assigned' with names of Borough
df2['Neighbourhood'] = np.where(df2['Neighbourhood'] == 'Not assigned',df2['Borough'], df2['Neighbourhood'])

df2

,Borough,Postal Code,Neighbourhood
0,North York,M3A,Parkwoods
1,North York,M4A,Victoria Village
2,Downtown Toronto,M5A,"Regent Park, Harbourfront"
3,North York,M6A,"Lawrence Manor, Lawrence Heights"
4,Queen's Park,M7A,Ontario Provincial Government
...,...,...,...
98,Etobicoke,M8X,"The Kingsway, Montgomery Road, Old Mill North"
99,Downtown Toronto,M4Y,Church and Wellesley
100,East Toronto Business,M7Y,Enclave of M4L
101,Etobicoke,M8Y,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [30]:
# Shape of data frame
df2.shape

(103, 3)

<h3>Importing the csv file conatining the latitudes and longitudes for various neighbourhoods in Canada</h3>

In [36]:
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head(12)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


### Merging 2 tables for getting the Latitudes & Longitudes for various Neighbourhoods:

In [37]:
df3 = pd.merge(df2,lat_lon,on='Postal Code')
df3.head(12)

,Borough,Postal Code,Neighbourhood,Latitude,Longitude
0,North York,M3A,Parkwoods,43.753259,-79.329656
1,North York,M4A,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,M5A,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,M6A,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Queen's Park,M7A,Ontario Provincial Government,43.662301,-79.389494
5,Etobicoke,M9A,Islington Avenue,43.667856,-79.532242
6,Scarborough,M1B,"Malvern, Rouge",43.806686,-79.194353
7,North York,M3B,Don Mills North,43.745906,-79.352188
8,East York,M4B,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,Downtown Toronto,M5B,"Garden District, Ryerson",43.657162,-79.378937


# Clustering and the plotting of the neighbourhoods

### All the rows from the data frame where boroughs contain the word Toronto:

In [38]:
df4 = df3[df3['Borough'].str.contains('Toronto',regex=False)]
df4

,Borough,Postal Code,Neighbourhood,Latitude,Longitude
2,Downtown Toronto,M5A,"Regent Park, Harbourfront",43.654260,-79.360636
9,Downtown Toronto,M5B,"Garden District, Ryerson",43.657162,-79.378937
15,Downtown Toronto,M5C,St. James Town,43.651494,-79.375418
19,East Toronto,M4E,The Beaches,43.676357,-79.293031
20,Downtown Toronto,M5E,Berczy Park,43.644771,-79.373306
24,Downtown Toronto,M5G,Central Bay Street,43.657952,-79.387383
25,Downtown Toronto,M6G,Christie,43.669542,-79.422564
30,Downtown Toronto,M5H,"Richmond, Adelaide, King",43.650571,-79.384568
31,West Toronto,M6H,"Dufferin, Dovercourt Village",43.669005,-79.442259
35,East York/East Toronto,M4J,The Danforth East,43.685347,-79.338106


# Visualizing all the Neighbourhoods using Folium

In [39]:
map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(df4['Latitude'],df4['Longitude'],df4['Borough'],df4['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

# KMeans Clustering

In [ ]:
k=5
toronto_clustering = df4.drop(['Postal Code','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
kmeans.labels_
df4.insert(['Cluster Labels'], axis=1, inplace=True)

In [45]:
# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df4['Latitude'], df4['Longitude'], df4['Neighbourhood'], df4['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters